In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master('local[*]').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a204bfbd325579d45c3ec7e507a6415281973534e8bb189153419c7506441055
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data=spark.read.json('/content/gdrive/MyDrive/PBDM project/Cell_Phones_and_Accessories_5.json')

In [ ]:
data.show()

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|7508492919| NULL|    5.0|Looks even better...| 08 4, 2014|A24E3SXTC62LJI|    Claudia Valdivia|{NULL,  Bling, NU...|Can't stop won't ...|    1407110400|    true|NULL|
|7508492919| NULL|    5.0|When you don't wa...|02 12, 2014|A269FLZCB4GIPV|         sarah ponce|                NULL|                   1|    1392163200|    true|NULL|
|7508492919| NULL|    3.0|so the case came ...| 02 8, 2014| AB6CHQWHZW4TV|                 Kai|                NULL|            Its okay|    1391817600|    true|NULL

In [ ]:
data=data.filter((data.reviewText.isNotNull()) & (data["summary"].isNotNull()))

In [ ]:
udata=data.dropDuplicates()

In [ ]:
udata.show()

+----------+-----+-------+--------------------+-----------+--------------+-----------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|     reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+-----------------+--------------------+--------------------+--------------+--------+----+
|8288862993| NULL|    5.0|           Excellent| 09 2, 2014|A26BTJ916KABRA|   Eneicy Paradas|                NULL|          Five Stars|    1409616000|    true|NULL|
|9707716371| NULL|    5.0|Item exactly as d...|09 13, 2017|A2PLVMA83745ZS|              gr2|                NULL|Item exactly as d...|    1505260800|    true|   2|
|9707716371| NULL|    5.0|Works perfect and...|04 21, 2017|A30B03YQN6SR1P|   Agentsmith2099|                NULL|            Perfect!|    1492732800|    true|NULL|
|9985538250| NUL

In [ ]:
cols_to_drop=['asin','image','style','vote']
udata=udata.drop(*cols_to_drop)

In [ ]:
udata.show()

In [ ]:
import pyspark.sql.functions as func
udata=udata.withColumn("reviewText_lower",func.lower(udata["reviewText"]))
udata=udata.withColumn("summary_lower",func.lower(udata["summary"]))

In [ ]:
udata=udata.filter(~udata["reviewText"].contains("case") &
                  ~udata["reviewText"].contains("protector") &
                  ~udata["reviewText"].contains("cover"))
udata=udata.filter(~udata["summary"].contains("case") &
                  ~udata["summary"].contains("protector") &
                  ~udata["summary"].contains("cover"))

In [ ]:
udata.count()

In [ ]:
udata.show()

In [ ]:
fdata=udata.filter(udata.verified==True)

In [ ]:
fdata.count()

In [ ]:
fdata.show()

In [ ]:
from pyspark.sql.functions import concat_ws, when
fdata = fdata.withColumn("category",
                         when(fdata['reviewText_lower'].like('%battery%'), "battery")
                         .when(fdata['reviewText_lower'].like('%display%'), "display")
                         .when(fdata['reviewText_lower'].like('%ram%'), "ram")
                         .when(fdata['reviewText_lower'].like('%connectivity%'), "connectivity")
                         .when(fdata['reviewText_lower'].like('%camera%'), "camera")
                         .when(fdata['reviewText_lower'].like('%storage%'), "storage")
                         .when(fdata['reviewText_lower'].like('%screen%'), "display")
                         .when(fdata['reviewText_lower'].like('%processor%'), "processor")
                         .when(fdata['reviewText_lower'].like('%cpu%'), "processor")
                         .when(fdata['reviewText_lower'].like('%weight%'), "weight")
                         .when(fdata['reviewText_lower'].like('%speaker%'), "speaker")
                         .when(fdata['reviewText_lower'].like('%software%'), "software")
                         .when(fdata['reviewText_lower'].like('%operating system%'), "software")
                         .otherwise("NA"))


In [ ]:

category_counts = fdata.groupBy("category").count().orderBy("category")
category_counts.show()


In [ ]:
fdata=fdata.filter(~(fdata['category']=="NA"))


In [ ]:
fdata.show()

+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+
|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|verified|    reviewText_lower|       summary_lower|category|
+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+
|    5.0|Item exactly as d...|09 13, 2017|A2PLVMA83745ZS|                 gr2|Item exactly as d...|    1505260800|    true|item exactly as d...|item exactly as d...| battery|
|    5.0|Works perfect and...|04 21, 2017|A30B03YQN6SR1P|      Agentsmith2099|            Perfect!|    1492732800|    true|works perfect and...|            perfect!| battery|
|    5.0|I have never had ...|02 28, 2006| A32LVMRTL0XF6|           Shaunspad|       Great Headset|    1141084800|    true|i 

In [ ]:
fdata=fdata.withColumn('reviewText_lower',func.regexp_replace(fdata['reviewText_lower'],"[^a-z ]", ""))
#fdata=fdata.withColumn('reviewText_lower',func.regexp_replace(fdata['reviewText_lower'],"[' ']",','))
fdata=fdata.withColumn('reviewText_lower',func.split(fdata["reviewText_lower"], " "))

In [ ]:
fdata.show()

+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+
|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|verified|    reviewText_lower|       summary_lower|category|
+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+
|    5.0|Item exactly as d...|09 13, 2017|A2PLVMA83745ZS|                 gr2|Item exactly as d...|    1505260800|    true|[item, exactly, a...|item exactly as d...| battery|
|    5.0|Works perfect and...|04 21, 2017|A30B03YQN6SR1P|      Agentsmith2099|            Perfect!|    1492732800|    true|[works, perfect, ...|            perfect!| battery|
|    5.0|I have never had ...|02 28, 2006| A32LVMRTL0XF6|           Shaunspad|       Great Headset|    1141084800|    true|[i

In [ ]:
from pyspark.ml.feature import StopWordsRemover
remover=StopWordsRemover(inputCol='reviewText_lower',outputCol='filtered')
n_df=remover.transform(fdata)

In [ ]:
n_df.show()

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentimental_Score(sentence):
    sentence=' '.join(sentence)
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentence)
    score=vs['compound']
    if score >= 0.5:
        return 'pos'+","+str(score)
    elif (score > -0.5) and (score < 0.5):
        return 'neu'+","+str(score)
    elif score <= -0.5:
        return 'neg'+","+str(score)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
from pyspark.sql.types import DoubleType,StringType
sentiment_udf=func.udf(sentimental_Score,StringType())

In [ ]:
#n_df['sentiment']=n_df['filtered'].map(sentimental_Score)
n_df=n_df.withColumn('sentiment',sentiment_udf(n_df['filtered']))

In [ ]:
n_df.show()

+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+--------------------+-----------+
|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|verified|    reviewText_lower|       summary_lower|category|            filtered|  sentiment|
+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+--------+--------------------+-----------+
|    5.0|Item exactly as d...|09 13, 2017|A2PLVMA83745ZS|                 gr2|Item exactly as d...|    1505260800|    true|[item, exactly, a...|item exactly as d...| battery|[item, exactly, d...| pos,0.7906|
|    5.0|Works perfect and...|04 21, 2017|A30B03YQN6SR1P|      Agentsmith2099|            Perfect!|    1492732800|    true|[works, perfect, ...|            perfect!| ba

In [ ]:
n_df.count()

In [ ]:
n_df= n_df.filter(func.size(n_df['filtered']) <= 50)

In [ ]:
n_df.count()

In [ ]:
n_df=n_df.withColumn("reviewText_lower_str",func.concat_ws(",", n_df["reviewText_lower"]))
n_df=n_df.drop("reviewText_lower")
n_df=n_df.withColumn("filtered_str",func.concat_ws(",", n_df["filtered"]))
n_df=n_df.drop("filtered")

In [ ]:
n_df.show()

+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------+-----------+--------------------+--------------------+
|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|verified|       summary_lower|category|  sentiment|reviewText_lower_str|        filtered_str|
+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------+--------------------+--------+-----------+--------------------+--------------------+
|    5.0|Works perfect and...|04 21, 2017|A30B03YQN6SR1P|      Agentsmith2099|            Perfect!|    1492732800|    true|            perfect!| battery| pos,0.5719|works,perfect,and...|works,perfect,las...|
|    3.0|The protective sh...|07 23, 2008|A1K2WS7DJ0X2DJ|          D. Johnson|Hmmm...not that g...|    1216771200|    true|hmmm...not that g...| display|neu,-0.1043|the

In [ ]:
#l_df=n_df.limit(100)

In [ ]:
#l_df.write.option("header",True).csv('/content/gdrive/MyDrive/PBDM project/cell_phone_analysis(final)2.csv',mode="overwrite")

In [ ]:
pdf=n_df.toPandas()

In [ ]:
pdf.head()

,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,verified,summary_lower,category,sentiment,reviewText_lower_str,filtered_str
0,5.0,Works perfect and it lasts significantly very ...,"04 21, 2017",A30B03YQN6SR1P,Agentsmith2099,Perfect!,1492732800,True,perfect!,battery,"pos,0.5719","works,perfect,and,it,lasts,significantly,very,...","works,perfect,lasts,significantly,long,battery..."
1,3.0,The protective sheet for the inside screen wor...,"07 23, 2008",A1K2WS7DJ0X2DJ,D. Johnson,Hmmm...not that great,1216771200,True,hmmm...not that great,display,"neu,-0.1043","the,protective,sheet,for,the,inside,screen,wor...","protective,sheet,inside,screen,works,great,out..."
2,2.0,LED light to indicate whether the batter was c...,"02 27, 2013",A12M4LP11ILM70,ET,LED Never Showed Fully Charged,1361923200,True,led never showed fully charged,battery,"neu,-0.2023","led,light,to,indicate,whether,the,batter,was,c...","led,light,indicate,whether,batter,charged,neve..."
3,3.0,Did not work with my HTC Eris Droid to well. I...,"04 8, 2010",A16JSCA4XNZDT4,James Ross,works ok. not for noisey enviroments.,1270684800,True,works ok. not for noisey enviroments.,battery,"pos,0.7269","did,not,work,with,my,htc,eris,droid,to,well,it...","work,htc,eris,droid,well,get,staticy,times,hea..."
4,2.0,I purchase this battery back in 2013 and use t...,"09 12, 2016",A34PLO79FZAECV,P. Kim,short shelf life.,1473638400,True,short shelf life.,battery,"pos,0.5853","i,purchase,this,battery,back,in,,and,use,two,w...","purchase,battery,back,,use,two,worked,well,try..."


In [ ]:
pdf.to_csv('/content/gdrive/MyDrive/PBDM project/results__.csv')